Dans cet autre cas pratique, nous utiliserons les bibliothèques python (Pandas, NumPy, Matplotlib) pour analyser des données sur **l'économie de carburant** pour différents modèles de véhicules de 2008 et 2018. Nous rencontrerons de nouveaux problèmes relatifs aux données et verrons différentes façons de les résoudre. Nous verrons également des techniques plus poussées pour manipuler et visualiser les données.
    

## Rappel sur le processus d'analyse de données :

### Questionnement --> Acquisition (Collecte, Évaluation, Nettoyage) --> Analyse (Statistiques, Visualisations)
###  --> Conclusions --> Partage des résultats

## 1. Aperçu des données

Les données relatives à l'économie de carburant proviennent des essais de véhicules effectués au laboratoire national des émissions des véhicules et des carburants de l'EPA à Ann Arbor, dans le Michigan. L'EPA fournit chaque année au gouvernement ces données que le ministère de l'énergie utilise pour publier son guide d'économie de carburant.

Attributs de chaque ensemble de données :

### Attributs

![Vehicule attributes](basics_images/vehicule_attributes1.png)
![Vehicule attributes](basics_images/vehicule_attributes2.png)

## 2. Poser des questions

- Y a-t-il une augmentation du nombre de modèles utilisant des sources de carburant alternatives ? De combien ? 
- Dans quelle mesure les classes de véhicules se sont-elles améliorées en matière d'économie de carburant ?
- Quelles sont les caractéristiques des véhicules Smartway ?
- Quelles sont les caractéristiques associées à une meilleure économie de carburant ?
- Pour tous les modèles qui ont été produits en 2008 et qui sont toujours en production en 2018, dans quelle mesure la valeur mpg s'est-elle améliorée et quel véhicule s'est-il le plus amélioré ?
- ...

## 3. Acquisition des données

### 3.1. Collecte des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

*Importer les données de 2008*

In [ ]:
df_08 = pd.read_csv('datasets/all_alpha_08.csv')
df_08.head()

*Importer les données de 2018*

In [ ]:
df_18 = pd.read_csv('datasets/all_alpha_18.csv')

In [ ]:
df_18.head()

### 3.2. Évaluation des données

Dans notre cas il s'agira de chercher les informations suivantes :
- nombre d'échantillons dans chaque ensemble de données
- nombre de colonnes dans chaque ensemble de données
- types de colonnes des données
- caractéristiques avec valeurs manquantes
- nombre de lignes dupliquées dans les deux jeux de données

###### Trouvez le nombre exact pour chacun des éléments suivants dans l'ensemble de données de 2008 :

Échantillons : 2404

Colonnes : 18

Lignes dupliquées : 25

Lignes avec des valeurs manquantes : ...

In [ ]:
df_08.info()

In [ ]:
df_08.shape

In [ ]:
df_08.duplicated().sum()

In [ ]:
df_08.isna().sum()

###### Trouvez le nombre exact pour chacun des éléments suivants dans l'ensemble de données de 2018 :

Échantillons : 1611

Colonnes : 18

Lignes dupliquées : ...

Lignes avec des valeurs manquantes : ...

In [ ]:
df_18.info()

In [ ]:
df_18.duplicated().sum()

In [ ]:
df_18.isna().sum()

##### Donner le type de données pour chaque caractéristique (certaines d'entre elles peuvent ne pas être idéales) :

Cyl (2018) : float

Cyl (2008) : object (chaîne de caractères)

MPG en ville (les deux) : object

Score de gaz à effet de serre (2008) : object

Score de gaz à effet de serre (2018) : int


In [ ]:
df_18["Cyl"].dtype

In [ ]:
df_18.Cyl.dtype

In [ ]:
df_08["Cyl"].dtype

##### Trouver le nombre de valeurs uniques non nulles pour chacune des caractéristiques suivantes :

SmartWay (2008) : 2

SmartWay (2018) : 3

Zone de vente (2008) : ...

Région de certification (2018) : ...

Transport (2008) : ...


In [ ]:
df_08['SmartWay'].unique()

In [ ]:
df_08['SmartWay'].nunique()

In [ ]:
df_08['SmartWay'].value_counts()

In [ ]:
df_18['SmartWay'].unique()

##### En quoi les colonnes « Cyl » des ensembles de données de 2008 et de 2018 diffèrent-elles ?

...

##### Dans quel jeu de données chacun de ces types de carburant est-il présent ?

Essence :

Éthanol/gaz :

Essence/électricité :

GNC :

Électricité :

### 3.3. Nettoyage des données

#### A) Nettoyer les libellés des colonnes

1. **Supprimer les colonnes superflues**

    Supprimer les caractéristiques qui ne sont pas uniformes (non présentes dans les deux ensembles de données) ou qui ne sont pas pertinentes pour nos questions. Utilisez la fonction drop de pandas.

    Colonnes à supprimer :

        De l'ensemble de données de 2008 : « Stnd », « Underhood ID », « FE Calc Appr », « Unadj Cmb MPG »
        De l'ensemble de données de 2018 : « Stnd », « Stnd Description », « Underhood ID », « Comb CO2 ».


In [ ]:
df_08.drop(columns=['Stnd', 'Underhood ID', 'FE Calc Appr', 'Unadj Cmb MPG'], inplace=True)

In [ ]:
df_18.drop(columns=['Stnd', 'Underhood ID', 'Stnd Description', 'Comb CO2'], inplace=True)

In [ ]:
df_08.columns

In [ ]:
df_18.columns

2. **Renommer les colonnes**

    Modifier le libellé de colonne « Sales Area » dans l'ensemble de données de 2008 pour « Cert Region » pour assurer l'uniformité.
    
    Renommez tous les libellés des colonnes pour remplacer les espaces par des caractères de soulignement et convertissez tout en minuscules.

In [ ]:
df_08.rename(columns={'Sales Area': 'Cert Region'}, inplace=True)

In [ ]:
df_08.columns == df_18.columns

In [ ]:
# remplacement des espaces par des caractères de soulignement et conversion en minuscules
df_08.rename(columns=lambda x: x.strip().replace(' ', '_').lower(), inplace=True)
df_18.rename(columns=lambda x: x.strip().replace(' ', '_').lower(), inplace=True)

In [ ]:
df_08.columns

In [ ]:
df_08.columns == df_18.columns

#### B) Filtrer, supprimer les valeurs nulles, supprimer les doublons

1. Filtrer

    Par souci d'uniformité, on va comparer seulement les voitures certifiées selon les normes californiennes. Filtrez les deux ensembles de données à l'aide de la fonction « query » pour sélectionner seulement les lignes où « cert_region » est « CA ». Ensuite, supprimez les colonnes « cert_region », car elles ne fourniront plus d'informations utiles (puisqu'on saura que chaque valeur est « CA »).

In [ ]:
df_08 = df_08.query('cert_region == "CA"')
df_08['cert_region'].unique()

In [ ]:
df_18 = df_18.query('cert_region == "CA"')
df_18['cert_region'].unique()

In [ ]:
df_08.drop(columns='cert_region', inplace=True)
df_18.drop(columns='cert_region', inplace=True)

2. Supprimer les valeurs nulles

    Supprimez les lignes des deux ensembles de données qui contiennent des valeurs manquantes.

In [ ]:
df_08.dropna(inplace=True)
df_18.dropna(inplace=True)

In [ ]:
df_08.isna().sum()

In [ ]:
df_18.isna().sum()

3. Supprimer les doublons

    Supprimez les éventuelles lignes dupliquées dans les deux ensembles de données.

In [ ]:
df_08.drop_duplicates(inplace=True)
df_18.drop_duplicates(inplace=True)

In [ ]:
df_08.duplicated().sum()

In [ ]:
df_18.duplicated().sum()

#### C) Corriger les types de données

**Corriger le type de donnée de « cyl »** :

    2008 : extraire la valeur entière de la chaîne.
    2018 : convertir la valeur flottante en valeur entière.


Voici le [lien](https://stackoverflow.com/questions/35376387/extract-int-from-string-in-pandas) qui m'a aidé à extraire la valeur entière de *cyl* pour 2008.

In [ ]:
df_08.info()

In [ ]:
# Extract int from strings in the 2008 cyl column
df_08['cyl'] = df_08['cyl'].str.extract('(\d+)').astype(int)

In [ ]:
df_08.cyl.dtype

In [ ]:
df_18['cyl'] = df_18['cyl'].astype(int)

In [ ]:
df_18.cyl.dtype

**Corriger le type de donnée de « air_pollution_score »**

    2008 : convertir la chaîne en valeur flottante.
    2018 : convertir la valeur entière en valeur flottante.


In [ ]:
df_08.air_pollution_score = df_08.air_pollution_score.astype('float')

In [ ]:
hb_08 = df_08[df_08['fuel'].str.contains('/')]
hb_18 = df_18[df_18['fuel'].str.contains('/')]

df1 = hb_08.copy()  
df2 = hb_08.copy()

# colonnes à séparer d'un "/"
split_columns = ['fuel', 'air_pollution_score', 'city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score']

# appliquer le split à chaque colonne de chaque copy de dataframe
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])
    
new_rows = df1.append(df2)

# supprimer les lignes des hybrides de départ
df_08.drop(hb_08.index, inplace=True)

# ajouter les lignes nouvellement obtenues
df_08 = df_08.append(new_rows, ignore_index=True)

On refait la même chose pour les données de 2018 :

In [ ]:
df1 = hb_18.copy()  
df2 = hb_18.copy()

# colonnes à séparer d'un "/"
split_columns = ['fuel', 'city_mpg', 'hwy_mpg', 'cmb_mpg']

# appliquer le split à chaque colonne de chaque copy de dataframe
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])
    
new_rows = df1.append(df2)

# supprimer les lignes des hybrides de départ
df_18.drop(hb_18.index, inplace=True)

# ajouter les lignes nouvellement obtenues
df_18 = df_18.append(new_rows, ignore_index=True)

On peut maintenant faire les changements voulus sans problème :

In [ ]:
df_08['air_pollution_score'] = df_08['air_pollution_score'].astype(float)
df_18['air_pollution_score'] = df_18['air_pollution_score'].astype(float)

In [ ]:
df_08.air_pollution_score.dtype

In [ ]:
df_18.air_pollution_score.dtype

**Corriger les types de données de « city_mpg », « hwy_mpg » et « cmb_mpg »**

    2008 et 2018 : convertir la chaîne en valeur flottante.


In [ ]:
for column in ['city_mpg', 'hwy_mpg', 'cmb_mpg']:
    df_08[column] = df_08[column].astype(float)
    df_18[column] = df_18[column].astype(float)

In [ ]:
df_08.info()

In [ ]:
df_18.info()

**Corriger le type de donnée de « greenhouse_gas_score »**

    2008 : convertir la valeur flottant en valeur entière.


In [ ]:
df_08.greenhouse_gas_score = df_08.greenhouse_gas_score.astype('int')

In [ ]:
df_08.greenhouse_gas_score.dtype

In [ ]:
df_08.dtypes == df_18.dtypes

## 4. Explorations avec des visualisations 

##### Comparer les distributions des scores de gaz à effet de serre en 2008 et 2018 :

In [ ]:
df_08.greenhouse_gas_score.hist();

In [ ]:
df_18.greenhouse_gas_score.hist();

##### Décrire la corrélation entre le déplacement et les mpg combinés :

In [ ]:
df_08.plot(x='displ', y='cmb_mpg', kind='scatter');

In [ ]:
df_18.plot(x='displ', y='cmb_mpg', kind='scatter');

##### Décrire la corrélation entre le score de gaz à effet de serre et les mpg combinés :

In [ ]:
df_08.columns

In [ ]:
df_08.plot(x='greenhouse_gas_score', y='cmb_mpg', kind='scatter');

In [ ]:
df_18.plot(x='greenhouse_gas_score', y='cmb_mpg', kind='scatter');

##### Y a-t-il plus de modèles uniques utilisant des sources de carburant alternatives ?

Regardons d'abord quelles sont les sources de carburant dans chaque jeu de données et lesquelles sont alternatives :

In [ ]:
df_08.fuel.value_counts()

In [ ]:
df_18.fuel.value_counts()

2008 : CNG, éthanol et 2018 : éthanol, électricité

In [ ]:
# combien de modèles uniques utilisaient des sources alternatives sources de carburant in 2008
alt_08 = df_08.query('fuel in ["CNG", "ethanol"]').model.nunique()
alt_08

In [ ]:
# combien de modèles uniques utilisaient des sources alternatives sources de carburant in 2018
alt_18 = df_18.query('fuel in ["Electricity", "Ethanol"]').model.nunique()
alt_18

In [ ]:
plt.bar(["2008", "2018"], [alt_08, alt_18])
plt.title("Number of Unique Models Using Alternative Fuels")
plt.xlabel("Year")
plt.ylabel("Number of Unique Models");

##### De combien les classes de véhicules ont-elles amélioré leur économie de carburant ?

In [ ]:
veh_08 = df_08.groupby('veh_class').cmb_mpg.mean()
veh_08

In [ ]:
veh_18 = df_18.groupby('veh_class').cmb_mpg.mean()
veh_18

In [ ]:
# quel accroissement chaque classe de véhicule a connu
acc = veh_18 - veh_08
acc

In [ ]:
# l'affichage ne concerne que les classes qui existent pour les deux années
acc.dropna(inplace=True)
plt.subplots(figsize=(8, 5))
plt.bar(acc.index, acc)
plt.title('Improvements in Fuel Economy from 2008 to 2018 by Vehicle Class')
plt.xlabel('Vehicle Class')
plt.ylabel('Increase in Average Combined MPG');

##### Pour tous les modèles produits en 2008 qui sont toujours en production en 2018, dans quelle mesure la valeur mpg s'est-elle améliorée et quel véhicule s'est le plus amélioré ?

In [ ]:
survivor_models = [model for model in df_08.model.unique() if model in df_18.model.value_counts()]
survivor_models

In [ ]:
survivors_08 = df_08.query(f'model in {survivor_models}')
survivors_08.shape

In [ ]:
survivors_18 = df_18.query(f'model in {survivor_models}')
survivors_18.shape

In [ ]:
mean_08 = survivors_08.groupby('model')[['cmb_mpg']].mean()
mean_08

In [ ]:
mean_18 = survivors_18.groupby('model')[['cmb_mpg']].mean()
mean_18

In [ ]:
ameliorations = mean_18 - mean_08
ameliorations.sort_values(by='cmb_mpg')